<a href="https://colab.research.google.com/github/TurkuNLP/Deep_Learning_in_LangTech_course/blob/master/hf_seq2seq_lemmatization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The task

* Lemmatization

# Data preparation

* Let us use universaldependencies.org
* 

In [8]:
!pip3 install --quiet datasets transformers
!wget -O train.conllu https://github.com/UniversalDependencies/UD_Finnish-TDT/raw/master/fi_tdt-ud-train.conllu
!wget -O validation.conllu https://github.com/UniversalDependencies/UD_Finnish-TDT/raw/master/fi_tdt-ud-dev.conllu
!wget -O test.conllu https://github.com/UniversalDependencies/UD_Finnish-TDT/raw/master/fi_tdt-ud-test.conllu

--2022-04-07 12:01:25--  https://github.com/UniversalDependencies/UD_Finnish-TDT/raw/master/fi_tdt-ud-train.conllu
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/UniversalDependencies/UD_Finnish-TDT/master/fi_tdt-ud-train.conllu [following]
--2022-04-07 12:01:25--  https://raw.githubusercontent.com/UniversalDependencies/UD_Finnish-TDT/master/fi_tdt-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13443822 (13M) [text/plain]
Saving to: ‘train.conllu’

train.conllu        100%[===================>]  12.82M  --.-KB/s    in 0.08s   

2022-04-07 12:01:25 (151 MB/s) - ‘train.conllu’ sa

In [9]:
import json

ID,FORM,LEMMA,UPOS,XPOS,FEAT,HEAD,DEPREL,DEPS,MISC=range(10)

def yield_examples(fname):
    with open(fname) as f:
        for line in f:
            line=line.rstrip("\n")
            if not line or line.startswith("#"):
                continue
            cols=line.split("\t")
            if not cols[0].isnumeric():
                continue
            form_tags,lemma=cols[FORM]+"_"+cols[UPOS]+"_"+cols[FEAT],cols[LEMMA]
            yield {"form_tags":form_tags,"lemma":lemma}

for fname in ("train.conllu","validation.conllu","test.conllu"):
    with open(fname.replace(".conllu",".jsonl"),"wt") as f_out:
        for example in yield_examples(fname):
            print(json.dumps(example),file=f_out)



In [10]:
import datasets

dataset = datasets.load_dataset(
    'json',                             # Format of the data
    data_files={"train":"train.jsonl","validation":"validation.jsonl","test":"test.jsonl"},
    split={
        "train":"train",  # First 80% is the train set
        "validation":"validation",   # Next 10% is the validation/dev set
        "test":"test"    # last 10% is the test set
    },
    features=datasets.Features({    # Here we tell how to interpret the attributes
        "form_tags":datasets.Value("string"),
        "lemma":datasets.Value("string")
    })
)


Using custom data configuration default-39a0dea4e99bd318


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-39a0dea4e99bd318/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['form_tags', 'lemma'],
        num_rows: 162816
    })
    validation: Dataset({
        features: ['form_tags', 'lemma'],
        num_rows: 18308
    })
    test: Dataset({
        features: ['form_tags', 'lemma'],
        num_rows: 21070
    })
})

# Tokenize and prep

In [12]:
import transformers

model_name = "TurkuNLP/bert-base-finnish-cased-v1"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)



Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/414k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/796k [00:00<?, ?B/s]

In [13]:
tokenizer("hei")

{'input_ids': [102, 2790, 103], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [14]:
def tokenize(example):
    inp_tok=tokenizer(example["form_tags"])
    outp_tok=tokenizer(example["lemma"])
    return {"input_ids":inp_tok["input_ids"],
            "attention_mask":inp_tok["attention_mask"],
            "token_type_ids":inp_tok["token_type_ids"],
            "output_input_ids":outp_tok["input_ids"],
            "output_attention_mask":outp_tok["attention_mask"],
            "output_token_type_ids":outp_tok["token_type_ids"]}

dataset=dataset.map(tokenize)

  0%|          | 0/162816 [00:00<?, ?ex/s]

  0%|          | 0/18308 [00:00<?, ?ex/s]

  0%|          | 0/21070 [00:00<?, ?ex/s]

In [15]:
config_encoder = transformers.BertConfig()
config_decoder = transformers.BertConfig()
config = transformers.EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)
model = transformers.EncoderDecoderModel(config=config)



In [17]:
help(model.forward)

Help on method forward in module transformers.models.encoder_decoder.modeling_encoder_decoder:

forward(input_ids=None, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, encoder_outputs=None, past_key_values=None, inputs_embeds=None, decoder_inputs_embeds=None, labels=None, use_cache=None, output_attentions=None, output_hidden_states=None, return_dict=None, **kwargs) method of transformers.models.encoder_decoder.modeling_encoder_decoder.EncoderDecoderModel instance
    The [`EncoderDecoderModel`] forward method, overrides the `__call__` special method.
    
    <Tip>
    
    Although the recipe for forward pass needs to be defined within this function, one should call the [`Module`]
    instance afterwards instead of this since the former takes care of running the pre and post processing steps while
    the latter silently ignores them.
    
    </Tip>
    
    Args:
        input_ids (`torch.LongTensor` of shape `(batch_size, sequence_length)`):
            In